In [1]:
# gpu 할당 확인
import tensorflow as tf

tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
# 이 아래에서부터 모든 셀 gpu로 할당
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

In [3]:
### General imports ###
import os
from glob import glob
import pickle
import numpy as np

### Plot imports ###
from IPython.display import Image
import matplotlib.pyplot as plt

### Time Distributed ConvNet imports ###
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, TimeDistributed, concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization, LeakyReLU, Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import backend as K
from keras.utils import np_utils
from keras.utils import plot_model
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import tensorflow as tf
from transformers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Warning ###
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = np.load('./speechEmotion1.npz')
X_train=data['X_train']
y_train=data['y_train']
y_test=data['y_test']
X_test=data['X_test']

In [ ]:
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(np.ravel(y_train)))
y_test = np_utils.to_categorical(lb.fit_transform(np.ravel(y_test)))


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train,
                                           y_train,random_state=42 ,test_size=0.25)
                                                             # 랜덤 고정 숫자

In [ ]:
# Reshape for convolution
X_train = X_train.reshape(X_train.shape[0], 3, 6, 10,1)
X_test = X_test.reshape(X_test.shape[0], 3, 6, 10,1)

In [ ]:
# Define two sets of inputs: MFCC and FBANK
input_y = Input(shape=X_train.shape[1:])

## First LFLB (local feature learning block)
y = Conv2D(16, kernel_size=(3, 3), strides=(1, 1), padding='same')(input_y)
y = BatchNormalization()(y)
y = Activation('elu')(y)
y = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(y)
y = Dropout(0.2)(y)     

## Second LFLB (local feature learning block)
y = Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('elu')(y)
y = MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same')(y)
y = Dropout(0.2)(y)

## Second LFLB (local feature learning block)
y = Conv2D(16, kernel_size=(3, 3), strides=(1, 1), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('elu')(y)
y = MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same')(y)
y = Dropout(0.2)(y)

## Second LFLB (local feature learning block)
y = Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same')(y)
y = BatchNormalization()(y)
y = Activation('elu')(y)
y = MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same')(y)
y = Dropout(0.2)(y)  

## Flat
y = Flatten()(y)                      
                               
# Apply 2 LSTM layer and one FC

y = Dense(y_train.shape[1], activation='softmax', name='FC')(y)

# Build final model
model = Model(inputs=input_y, outputs=y)


In [ ]:
# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Save best model
# best_model_save = ModelCheckpoint('./[CNN-LSTM]Model.hdf5', save_best_only=True, monitor='val_acc', mode='max')

# # Early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=30, verbose=1, mode='max')

# Fit model
history = model.fit(X_train, y_train, batch_size=64, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping])

In [ ]:
model.save('voice_emotion_CnnModel1008.h5')